In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
from pulp import LpProblem, LpVariable, lpSum, LpMinimize, LpStatus
from shapely.geometry import Point

In [2]:
df = gpd.read_file("data.geojson")

In [4]:
df_colegios = pd.read_excel("predata/definitivo.xlsx")
df_colegios

,CODIGO_DANE,COLE_INST_NOMBRE,SECTOR,COLE_CATEGORIA,TOTAL_MATRICULA
0,305001025045,C.E. PASITOS TRAVIESOS,NO_OFICIAL,1,48
1,305001025037,C.E. SEMILLAS DE ESPERANZA,NO_OFICIAL,1,4
2,305001025665,PEESCOLAR HERMANOS GRIMM S,NO_OFICIAL,1,32
3,305001025690,JARDÃƒÂN INFANTIL MÃƒÂGICO UNIVERSO,NO_OFICIAL,2,34
4,305001025711,CENT EDUC REVIVIR,NO_OFICIAL,1,144
...,...,...,...,...,...
598,305001021261,COL GENTE UNIDA JOVENES POR LA PAZ - MORAVIA,NO_OFICIAL,5,1703
599,305001022232,INST EDUC BARRIO OLAYA HERRERA,OFICIAL,1,1540
600,305001022461,COL GENTE UNIDA JOVENES POR LA PAZ - LUZ DE OR...,NO_OFICIAL,2,542
601,305001022631,CORP ESC EMPRESARIAL DE EDUCACION,NO_OFICIAL,2,2664


In [5]:
calidad = df_colegios['COLE_CATEGORIA'].repeat(10).tolist()
capacidad = df_colegios['TOTAL_MATRICULA'].repeat(10).tolist()
demanda = df['POBLACION_MANZANA'].iloc[:800].tolist()

In [6]:
latitudes = np.random.uniform(6.1925, 6.35, len(df_colegios)*10)
longitudes = np.random.uniform(-75.6461, -75.5314, len(df_colegios)*10)
coordenadas = [Point(lon, lat) for lon, lat in zip(longitudes, latitudes)]

gdf_puntos = gpd.GeoDataFrame(geometry=coordenadas, crs=df.crs)

m_distancias = []

for idx, manzana in df.head(800).iterrows():
    distancias_manzana = []
    j=0
    for punto in gdf_puntos.geometry:
        distancia = punto.distance(manzana.geometry)
        distancias_manzana.append(distancia*1000-manzana.ESTRATO*calidad[j])
        j=j+1

    m_distancias.append(distancias_manzana)

# Convertir la matriz de distancias en un numpy array
distancias = np.array(m_distancias)


In [7]:
#distancias = np.loadtxt("distancias.txt")

In [8]:
num_schools=len(df_colegios)
num_possible_locations_per_school = 10

# Simulated annealing

In [9]:
from pulp import LpMinimize, LpProblem, LpVariable, lpSum

# Función objetivo: minimizar las distancias ponderadas entre los colegios y los estudiantes
def objective_function(selected_indices, distance_matrix, demand, capacity):
    # Crear un problema de optimización
    print("Buenas")
    prob = LpProblem("Optimal_Location", LpMinimize)
    
    # Variables binarias para representar si se selecciona un colegio o no
    selected = LpVariable.dicts("Selected", selected_indices, cat='Binary')
    print("tardes")
    # Variables de asignación: proporción de demanda asignada a cada colegio
    assignment_vars = LpVariable.dicts("Assignment", [(i, j) for i in range(len(demand)) for j in selected_indices], lowBound=0, upBound=1)
    print(",")
    # Función objetivo: minimizar las distancias ponderadas
    prob += lpSum(assignment_vars[(i, j)] * distance_matrix[i, j] for i in range(len(demand)) for j in selected_indices)
    print("amigos")
    # Restricción: proporción de asignación del punto de demanda i debe sumar 1
    for i in range(len(demand)):
        prob += lpSum(assignment_vars[(i, j)] for j in selected_indices) == 1
    print("¿Cómo")
    # Restricción: la suma de la demanda asignada a un colegio no debe exceder su capacidad
    for j in selected_indices:
        prob += lpSum(assignment_vars[(i, j)] * demand[i] for i in range(len(demand))) <= capacity[j] * selected[j]
    print("están")
    # Restricción: x_i_j <= y_j
    for i in range(len(demand)):
        for j in selected_indices:
            prob += assignment_vars[(i, j)] <= selected[j]
    print("?")
    # Resolver el problema de optimización
    prob.solve()
    print("!!")
    # Devolver el valor objetivo (suma de las distancias ponderadas)
    return prob.objective.value(), [selected[j].varValue for j in selected_indices]

# Generar solución inicial: seleccionar una ubicación de cada grupo de 10 posibles para cada colegio
def generate_initial_state(num_schools=num_schools, num_possible_locations_per_school=num_possible_locations_per_school):
    selected_indices = []
    for i in range(num_schools):
        # Selección aleatoria de una ubicación dentro del rango correspondiente de 10 posibles ubicaciones
        selected_indices.append(i * num_possible_locations_per_school + np.random.randint(num_possible_locations_per_school))
    return selected_indices

# Generar vecino: cambiar la ubicación de un colegio seleccionado por otra dentro del rango correspondiente
def generate_neighbor(current_state, num_possible_locations_per_school=num_possible_locations_per_school):
    new_state = current_state.copy()
    # Seleccionar un colegio aleatorio para perturbar
    index_to_perturb = np.random.randint(len(current_state))
    # Generar un nuevo índice dentro del rango correspondiente
    group_start = (current_state[index_to_perturb] // num_possible_locations_per_school) * num_possible_locations_per_school
    b=group_start + np.random.randint(num_possible_locations_per_school)
    while b==current_state[index_to_perturb]:
        b=group_start + np.random.randint(num_possible_locations_per_school)
    new_state[index_to_perturb] = b
    return new_state

# Simulated annealing algorithm
def simulated_annealing(distance_matrix, demand, capacity, max_iterations=1000, initial_temperature=1.0, cooling_rate=0.95):
    current_state = generate_initial_state()
    current_cost, current_selected = objective_function(current_state, distance_matrix, demand, capacity)
    best_state = current_selected
    best_cost = current_cost

    for iteration in range(max_iterations):
        temperature = initial_temperature * (cooling_rate ** iteration)
        new_state = generate_neighbor(current_state)
        print(new_state)
        new_cost, new_selected = objective_function(new_state, distance_matrix, demand, capacity)
        if new_cost < current_cost or np.random.rand() < acceptance_probability(current_cost, new_cost, temperature):
            current_state = new_state
            current_cost = new_cost
            if new_cost < best_cost:
                best_state = current_state
                best_cost = new_cost

    return best_state, best_cost

# Acceptance probability function
def acceptance_probability(old_cost, new_cost, temperature):
    if new_cost < old_cost:
        return 1.0
    return np.exp((old_cost - new_cost) / temperature)


In [10]:
best_state, best_cost = simulated_annealing(distancias, demanda, capacidad)

print("Best state:", best_state)
print("Best cost:", best_cost)

Buenas
tardes
,
amigos
¿Cómo
están
?
!!
[9, 17, 23, 33, 48, 54, 65, 70, 84, 90, 106, 114, 126, 132, 147, 151, 161, 172, 182, 190, 209, 218, 223, 233, 248, 255, 265, 271, 284, 291, 304, 312, 325, 330, 345, 356, 369, 374, 382, 390, 401, 416, 420, 437, 440, 451, 463, 475, 488, 493, 501, 510, 520, 534, 540, 557, 564, 575, 589, 590, 606, 616, 622, 633, 642, 658, 663, 675, 688, 697, 704, 713, 724, 737, 749, 752, 762, 770, 789, 790, 805, 811, 827, 831, 849, 851, 867, 878, 882, 898, 902, 917, 923, 936, 948, 954, 966, 973, 984, 992, 1009, 1018, 1028, 1034, 1041, 1058, 1064, 1079, 1084, 1092, 1109, 1112, 1121, 1139, 1146, 1154, 1165, 1173, 1185, 1190, 1207, 1219, 1227, 1239, 1248, 1257, 1267, 1274, 1287, 1297, 1304, 1313, 1321, 1335, 1346, 1352, 1364, 1373, 1381, 1399, 1409, 1411, 1425, 1432, 1442, 1452, 1461, 1479, 1484, 1490, 1505, 1515, 1526, 1534, 1549, 1554, 1563, 1576, 1582, 1595, 1608, 1619, 1628, 1633, 1642, 1653, 1664, 1678, 1687, 1698, 1700, 1718, 1724, 1735, 1744, 1754, 1763, 1772, 17

KeyboardInterrupt: 